# Plan: SIIM-FISABIO-RSNA COVID-19 Detection

Objectives:
- Verify GPU availability and environment.
- Inspect provided artifacts (CSV schemas, train/test structure, sample_submission).
- Establish a fast, reproducible CV protocol mirroring leaderboard metric.
- Build a minimal baseline submission ASAP; iterate with improvements.
- Request expert review after planning and after baseline.

Key questions for experts:
- What medal-winning strategies used for this comp (architectures, resolutions, losses, WBF/NMS details, augmentations, class-imbalance handling, CV protocol)?
- Common pitfalls (study vs image-level leakage, box formatting, confidence thresholds, resizing DICOMs/PNGs, metadata leakage)?

Immediate steps:
1) Env check (GPU, torch versions).
2) Explore CSVs: train_image_level.csv, train_study_level.csv, sample_submission.csv.
3) Confirm test set structure and submission format constraints.
4) If raw images not present, determine fallback (dummy boxes vs class-only predictions) matching sample_submission format.
5) Create robust pipeline skeleton (dataset loader, transforms, detector head for image-level boxes, classifier head for study-level labels).
6) Establish CV split by study, cache features, and log OOF metrics.

We will request expert review right after this planning + EDA cell runs.

In [1]:
import os, sys, subprocess, time, shutil, json, glob, textwrap
import pandas as pd
from pathlib import Path

def sh(cmd):
    print(f"$ {cmd}", flush=True)
    res = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(res.stdout, flush=True)
    return res.returncode

print("=== Env Check ===", flush=True)
sh("nvidia-smi || true")
print("Python:", sys.version)

print("\n=== Repo Files ===", flush=True)
for p in sorted(Path('.').glob('*')):
    try:
        print(p, ("dir" if p.is_dir() else f"{p.stat().st_size} B"))
    except Exception as e:
        print(p, e)

print("\n=== Data Snapshot ===", flush=True)
for fname in ["sample_submission.csv", "train_image_level.csv", "train_study_level.csv"]:
    if Path(fname).exists():
        df = pd.read_csv(fname)
        print(f"{fname}: shape={df.shape}")
        print(df.head(3))
        print("cols:", list(df.columns))
        print("-")
    else:
        print(f"Missing {fname}")

def count_files(root):
    root = Path(root)
    n_dirs = 0
    n_files = 0
    for dp, dns, fns in os.walk(root):
        n_dirs += len(dns)
        n_files += len(fns)
    return n_dirs, n_files

for d in ["train", "test"]:
    if Path(d).exists():
        nd, nf = count_files(d)
        print(f"{d}/ -> {nd} subdirs, {nf} files")
    else:
        print(f"Missing dir: {d}")

print("\nNote: If images are absent, we will construct a valid dummy submission matching sample_submission format as a baseline.")

=== Env Check ===


$ nvidia-smi || true


Thu Sep 25 02:40:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Python: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]

=== Repo Files ===


.00_eda_and_planning_kernel_state.json 183 B
00_eda_and_planning.ipynb 4448 B
agent_metadata dir
description.md 27318 B
docker_run.log 25775 B
requirements.txt 2021 B
sample_submission.csv 44740 B
submission.csv 44740 B
task.txt 3500 B
test dir
train dir
train_image_level.csv 1142993 B
train_study_level.csv 147186 B

=== Data Snapshot ===


sample_submission.csv: shape=(1244, 2)
                   id    PredictionString
0  000c9c05fd14_study  negative 1 0 0 1 1
1  00c74279c5b7_study  negative 1 0 0 1 1
2  00ccd633fb0e_study  negative 1 0 0 1 1
cols: ['id', 'PredictionString']
-
train_image_level.csv: shape=(5696, 4)
                   id                                              boxes  \
0  000a312787f2_image  [{'x': 789.28836, 'y': 582.43035, 'width': 102...   
1  000c3a3f293f_image                                                NaN   
2  0012ff7358bc_image  [{'x': 677.42216, 'y': 197.97662, 'width': 867...   

                                               label StudyInstanceUID  
0  opacity 1 789.28836 582.43035 1815.94498 2499....     5776db0cec75  
1                                     none 1 0 0 1 1     ff0879eb20ed  
2  opacity 1 677.42216 197.97662 1545.21983 1197....     9d514ce429a7  
cols: ['id', 'boxes', 'label', 'StudyInstanceUID']
-
train_study_level.csv: shape=(5448, 5)
                   id  Negative fo

test/ -> 1244 subdirs, 638 files

Note: If images are absent, we will construct a valid dummy submission matching sample_submission format as a baseline.


In [2]:
import pandas as pd
from pathlib import Path
import sys
print("=== Baseline Submission: filling by id suffix ===", flush=True)
sub_path = Path("sample_submission.csv")
assert sub_path.exists(), "sample_submission.csv not found"
sub = pd.read_csv(sub_path)
print("Sample sub shape:", sub.shape)

def fill_pred(row_id: str) -> str:
    return "negative 1 0 0 1 1" if str(row_id).endswith("_study") else "none 1 0 0 1 1"

sub["PredictionString"] = sub["id"].astype(str).apply(fill_pred)
out_path = Path("submission.csv")
sub.to_csv(out_path, index=False)
print("Wrote:", out_path.resolve())
print(sub.head(5))
print("Counts:", {"rows": len(sub), "study_rows": sub["id"].str.endswith("_study").sum(), "image_rows": sub["id"].str.endswith("_image").sum()})
assert sub["PredictionString"].astype(str).str.len().min() > 0, "Empty PredictionString detected"
print("Baseline submission ready." )

=== Baseline Submission: filling by id suffix ===


Sample sub shape: (1244, 2)
Wrote: /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-015252/submission.csv
                   id    PredictionString
0  000c9c05fd14_study  negative 1 0 0 1 1
1  00c74279c5b7_study  negative 1 0 0 1 1
2  00ccd633fb0e_study  negative 1 0 0 1 1
3  00e936c58da6_study  negative 1 0 0 1 1
4  01206a422293_study  negative 1 0 0 1 1
Counts: {'rows': 1244, 'study_rows': 606, 'image_rows': 638}
Baseline submission ready.


In [5]:
import os, glob, itertools, hashlib
from pathlib import Path
import pandas as pd
print("=== Locate image files recursively ===", flush=True)
exts = ("*.dcm", "*.dicom", "*.png", "*.jpg", "*.jpeg")
train_files = list(itertools.islice(itertools.chain.from_iterable(Path("train").rglob(pat) for pat in exts), 50))
test_files = list(itertools.islice(itertools.chain.from_iterable(Path("test").rglob(pat) for pat in exts), 50))
print("train sample count (up to 50):", len(train_files))
for p in train_files[:10]:
    try:
        st = p.stat()
        print("TRAIN:", p, st.st_size, "B")
    except Exception as e:
        print("TRAIN:", p, e)
print("test sample count (up to 50):", len(test_files))
for p in test_files[:10]:
    try:
        st = p.stat()
        print("TEST:", p, st.st_size, "B")
    except Exception as e:
        print("TEST:", p, e)

print("\n=== Try reading one train image ===", flush=True)
img_path = None
for p in train_files:
    if p.suffix.lower() in (".png", ".jpg", ".jpeg", ".dcm", ".dicom"):
        img_path = p
        break
print("Chosen:", img_path)
if img_path is None:
    print("No image file found under train with known extensions.")
else:
    try:
        if img_path.suffix.lower() in (".dcm", ".dicom"):
            import pydicom
            ds = pydicom.dcmread(str(img_path))
            arr = ds.pixel_array
            print("DICOM shape:", getattr(arr, 'shape', None), "dtype:", getattr(arr, 'dtype', None), "MONOCHROME1:", getattr(ds, 'PhotometricInterpretation', None) == 'MONOCHROME1')
        else:
            import cv2
            import numpy as np
            img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
            print("IMG shape:", None if img is None else img.shape, "dtype:", None if img is None else img.dtype)
    except Exception as e:
        print("Read error:", e)

print("\n=== Sanity: map one CSV id to a plausible path ===", flush=True)
img_df = pd.read_csv("train_image_level.csv")
sample_id = img_df.loc[0, "id"].replace("_image", "")
print("sample image id:", sample_id)
cand = list(Path("train").rglob(f"**/{sample_id}.*"))
print("candidate paths found:", len(cand))
for p in cand[:5]: print("CAND:", p)

=== Locate image files recursively ===


train sample count (up to 50): 50
TRAIN: train/14204d559d6c/2603e1c933cf/061067e90779.dcm 15675652 B
TRAIN: train/52ed9d91bb19/ff41637c1297/279e73ff65b6.dcm 29511640 B
TRAIN: train/fdcc6f504d49/7d8b6ee6143c/8529395d2a79.dcm 6571484 B
TRAIN: train/815278b6c176/d8c388c5305e/e463e244a127.dcm 6571550 B
TRAIN: train/accb3f0a6276/6edba3a25a28/70248cd9577f.dcm 6571484 B
TRAIN: train/0949b847bda6/899f0001cba9/e60a47d36e62.dcm 29567322 B
TRAIN: train/0949b847bda6/3c68f090d142/51df7e6104bb.dcm 13141724 B
TRAIN: train/19747df96950/18ef4c08508f/e5d02148b87b.dcm 15489208 B
TRAIN: train/edaf69ec10f6/121ce5da9efc/823797c902b5.dcm 14631644 B
TRAIN: train/41b86a3012d5/365e1e9b003f/5c8f5e53c8e8.dcm 13141722 B
test sample count (up to 50): 50
TEST: test/30e45593ba08/c5c0a57e6e4c/004cbd797cd1.dcm 17899398 B
TEST: test/e1a37ecaabfe/83877a524aaa/5d062f8686ff.dcm 15489208 B
TEST: test/00c74279c5b7/ca867739fd1b/136af218f8df.dcm 15698060 B
TEST: test/c65ed6750934/c2e3588fb042/7be323e5f816.dcm 13251038 B
TEST: 

Chosen: train/14204d559d6c/2603e1c933cf/061067e90779.dcm
DICOM shape: (2801, 2798) dtype: uint16 MONOCHROME1: False

=== Sanity: map one CSV id to a plausible path ===


sample image id: 000a312787f2


candidate paths found: 1
CAND: train/5776db0cec75/81456c9c5423/000a312787f2.dcm


In [4]:
import os, sys, subprocess, shutil, time
from pathlib import Path

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

# 0) Uninstall any existing torch stack (idempotent)
for pkg in ("torch","torchvision","torchaudio"):
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    "/app/.pip-target/torch",
    "/app/.pip-target/torch-2.8.0.dist-info",
    "/app/.pip-target/torch-2.4.1.dist-info",
    "/app/.pip-target/torchvision",
    "/app/.pip-target/torchvision-0.23.0.dist-info",
    "/app/.pip-target/torchvision-0.19.1.dist-info",
    "/app/.pip-target/torchaudio",
    "/app/.pip-target/torchaudio-2.8.0.dist-info",
    "/app/.pip-target/torchaudio-2.4.1.dist-info",
    "/app/.pip-target/torchgen",
    "/app/.pip-target/functorch",
):
    if os.path.exists(d):
        print("Removing", d)
        shutil.rmtree(d, ignore_errors=True)

print("Installing cu121 torch stack...", flush=True)
pip("install",
    "--index-url", "https://download.pytorch.org/whl/cu121",
    "--extra-index-url", "https://pypi.org/simple",
    "torch==2.4.1", "torchvision==0.19.1", "torchaudio==2.4.1")

Path("constraints.txt").write_text(
    "torch==2.4.1\n"
    "torchvision==0.19.1\n"
    "torchaudio==2.4.1\n"
)

print("Installing supporting libs...", flush=True)
pip("install", "-c", "constraints.txt",
    "timm==1.0.9",
    "albumentations==1.4.15",
    "opencv-python-headless==4.10.0.84",
    "pydicom==2.4.4",
    "numpy<2.0.0",
    "pandas",
    "scikit-learn",
    "pycocotools",
    "--upgrade-strategy", "only-if-needed")

import torch
print("torch:", torch.__version__, "built CUDA:", getattr(torch.version, "cuda", None))
print("CUDA available:", torch.cuda.is_available())
assert str(getattr(torch.version, "cuda", "")).startswith("12.1"), f"Wrong CUDA build: {torch.version.cuda}"
assert torch.cuda.is_available(), "CUDA not available"
print("GPU:", torch.cuda.get_device_name(0))
print("Setup complete.")

Installing cu121 torch stack...


> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 546.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 275.5 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 134.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 162.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 458.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 371.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 446.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 492.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 419.5 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
Setup complete.


In [10]:
import os, math, json, gc
from pathlib import Path
import numpy as np
import pandas as pd
import pydicom
import cv2
from concurrent.futures import ThreadPoolExecutor, as_completed

print("=== DICOM -> PNG cache (p2-p98, slope/intercept, MONOCHROME1, optional CLAHE) ===", flush=True)
cache_root = Path("cache")
cache_train = cache_root/"train"
cache_test = cache_root/"test"
cache_root.mkdir(parents=True, exist_ok=True)
cache_train.mkdir(parents=True, exist_ok=True)
cache_test.mkdir(parents=True, exist_ok=True)

def read_dicom(path: Path):
    ds = pydicom.dcmread(str(path))
    arr = ds.pixel_array.astype(np.float32)
    # rescale
    slope = float(getattr(ds, 'RescaleSlope', 1))
    intercept = float(getattr(ds, 'RescaleIntercept', 0))
    arr = arr * slope + intercept
    # invert if MONOCHROME1
    if getattr(ds, 'PhotometricInterpretation', '') == 'MONOCHROME1':
        # invert after rescale
        arr = -arr
    return arr, ds

def to_uint8_window(arr: np.ndarray, p_low=2, p_high=98, clahe=False):
    lo = np.percentile(arr, p_low)
    hi = np.percentile(arr, p_high)
    if hi <= lo:
        hi = lo + 1.0
    arr = np.clip((arr - lo) / (hi - lo), 0, 1)
    img = (arr * 255.0).astype(np.uint8)
    if clahe:
        cla = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = cla.apply(img)
    return img

def build_stem_map(root: Path):
    stem2path = {}
    for p in root.rglob("*.dcm"):
        stem2path[p.stem] = p
    return stem2path

print("Indexing DICOM paths...", flush=True)
train_map = build_stem_map(Path("train"))
test_map = build_stem_map(Path("test"))
print(f"train_map: {len(train_map)} dcm stems, test_map: {len(test_map)} dcm stems")

# Collect target ids
train_img_df = pd.read_csv("train_image_level.csv")
train_img_df["image_id"] = train_img_df["id"].str.replace("_image", "", regex=False)
sub = pd.read_csv("sample_submission.csv")
test_img_ids = sub.loc[sub["id"].str.endswith("_image"), "id"].str.replace("_image", "", regex=False).tolist()

meta_rows = []

def process_one(stem: str, is_train=True, clahe=False):
    mp = train_map if is_train else test_map
    out_dir = cache_train if is_train else cache_test
    p = mp.get(stem)
    if p is None:
        return None
    out_path = out_dir / f"{stem}.png"
    if out_path.exists():
        # we still want shape; read once using cv2
        try:
            ds = pydicom.dcmread(str(p), stop_before_pixels=True)
            h = int(getattr(ds, 'Rows', 0) or 0)
            w = int(getattr(ds, 'Columns', 0) or 0)
        except Exception:
            h = w = 0
        return (stem, h, w, str(out_path))
    try:
        arr, ds = read_dicom(p)
        h, w = arr.shape[:2]
        img = to_uint8_window(arr, p_low=2, p_high=98, clahe=clahe)
        # save as 1-channel PNG
        ok = cv2.imwrite(str(out_path), img)
        if not ok:
            return None
        return (stem, h, w, str(out_path))
    except Exception as e:
        print("Err:", stem, e, flush=True)
        return None

def run_pool(stems, is_train=True, workers=8, clahe=False):
    results = []
    with ThreadPoolExecutor(max_workers=workers) as ex:
        futs = {ex.submit(process_one, s, is_train, clahe): s for s in stems}
        done = 0
        total = len(futs)
        for fut in as_completed(futs):
            r = fut.result()
            if r is not None:
                results.append(r)
            done += 1
            if done % 100 == 0 or done == total:
                print(f"Processed {done}/{total}", flush=True)
    return results

# Determine which train stems to cache (intersection with train_map)
train_stems = [s for s in train_img_df["image_id"].tolist() if s in train_map]
test_stems = [s for s in test_img_ids if s in test_map]
print(f"To cache: train {len(train_stems)}, test {len(test_stems)}", flush=True)

# Run caching
train_res = run_pool(train_stems, is_train=True, workers=8, clahe=False)
test_res = run_pool(test_stems, is_train=False, workers=8, clahe=False)

meta = pd.DataFrame(train_res + test_res, columns=["image_id","orig_h","orig_w","png_path"])
meta_path = cache_root/"meta_image_shapes.csv"
meta.to_csv(meta_path, index=False)
print("Cached PNGs and wrote:", meta_path, "rows=", len(meta))
print(meta.head())
print("Cache complete.")

=== DICOM -> PNG cache (p2-p98, slope/intercept, MONOCHROME1, optional CLAHE) ===


Indexing DICOM paths...


train_map: 5696 dcm stems, test_map: 638 dcm stems
To cache: train 5696, test 638


Processed 100/5696


Err: 0452c373f62b The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. ), pylibjpeg (req. )


...

 e1f510dd50d5 The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. ), pylibjpeg (req. )


Err: e350c6a1acbd The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. ), pylibjpeg (req. )


Processed 5100/5696


Err: e3ab2d72cac2 The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. ), pylibjpeg (req. )


Err: e563f54f421c The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. ), pylibjpeg (req. )


In [7]:
import sys, subprocess
def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

print("Installing DICOM decompression plugins (pylibjpeg stack)...", flush=True)
pip("install", "pylibjpeg==2.0.2", "pylibjpeg-libjpeg==2.1.0", "pylibjpeg-openjpeg==2.3.0", "pylibjpeg-rle==2.0.0")
print("Verifying pydicom pixel data handlers...", flush=True)
import pydicom
from pydicom import config as pydicom_config
print("Available handlers:", pydicom_config.pixel_data_handlers, flush=True)
print("DICOM plugins installed.")

In [9]:
import sys, subprocess
def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

print("Installing GDCM backend for DICOM (python-gdcm)...", flush=True)
pip("install", "python-gdcm==3.0.22")
print("Verifying pydicom pixel data handlers after GDCM install...", flush=True)
import pydicom
from pydicom import config as pydicom_config
print("Available handlers:", pydicom_config.pixel_data_handlers, flush=True)
print("GDCM install complete.")

Installing GDCM backend for DICOM (python-gdcm)...


> install python-gdcm==3.0.22


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 11.2 MB/s eta 0:00:00


Verifying pydicom pixel data handlers after GDCM install...


Available handlers: [<module 'pydicom.pixel_data_handlers.numpy_handler' from '/app/.pip-target/pydicom/pixel_data_handlers/numpy_handler.py'>, <module 'pydicom.pixel_data_handlers.gdcm_handler' from '/app/.pip-target/pydicom/pixel_data_handlers/gdcm_handler.py'>, <module 'pydicom.pixel_data_handlers.pillow_handler' from '/app/.pip-target/pydicom/pixel_data_handlers/pillow_handler.py'>, <module 'pydicom.pixel_data_handlers.jpeg_ls_handler' from '/app/.pip-target/pydicom/pixel_data_handlers/jpeg_ls_handler.py'>, <module 'pydicom.pixel_data_handlers.pylibjpeg_handler' from '/app/.pip-target/pydicom/pixel_data_handlers/pylibjpeg_handler.py'>, <module 'pydicom.pixel_data_handlers.rle_handler' from '/app/.pip-target/pydicom/pixel_data_handlers/rle_handler.py'>]


GDCM install complete.


In [11]:
import os
from pathlib import Path
import pydicom
from pydicom.uid import UID
from pydicom import config as pydicom_config
import traceback
from collections import Counter

print("=== DICOM decompression debug ===", flush=True)
print("Handlers registered:")
for h in pydicom_config.pixel_data_handlers:
    print(" -", getattr(h, "__name__", str(h)))

def scan_transfer_syntax(root: str, limit=500):
    ts_counts = Counter()
    samples = {}  # ts -> path
    n = 0
    for p in Path(root).rglob("*.dcm"):
        try:
            ds = pydicom.dcmread(str(p), stop_before_pixels=True, force=True)
            ts = str(getattr(ds.file_meta, "TransferSyntaxUID", ""))
            if not ts:
                ts = "<missing>"
            ts_counts[ts] += 1
            if ts not in samples:
                samples[ts] = str(p)
        except Exception:
            ts_counts["<read_error>"] += 1
        n += 1
        if n >= limit:
            break
    return ts_counts, samples

train_ts_counts, train_ts_samples = scan_transfer_syntax("train", limit=2000)
test_ts_counts, test_ts_samples = scan_transfer_syntax("test", limit=2000)
print("Train TS counts (sampled):", dict(train_ts_counts))
print("Test  TS counts (sampled):", dict(test_ts_counts))

def try_decode(path: str):
    print(f"\n-- Try decode: {path}")
    ds = pydicom.dcmread(path, force=True)
    ts = str(getattr(ds.file_meta, "TransferSyntaxUID", ""))
    print("TransferSyntaxUID:", ts, UID(ts).name if ts else "<unknown>")
    print("PhotometricInterpretation:", getattr(ds, 'PhotometricInterpretation', None))
    print("Rows x Cols:", getattr(ds, 'Rows', None), getattr(ds, 'Columns', None))
    # Attempt via pixel_array
    try:
        arr = ds.pixel_array
        print("pixel_array OK:", getattr(arr, 'shape', None), getattr(arr, 'dtype', None))
        return True
    except Exception as e:
        print("pixel_array failed:", repr(e))
        # Try explicit ds.decompress (gdcm only)
        try:
            if hasattr(ds, 'decompress'):
                ds.decompress()
                arr2 = ds.pixel_array
                print("decompress() + pixel_array OK:", getattr(arr2, 'shape', None), getattr(arr2, 'dtype', None))
                return True
        except Exception as e2:
            print("decompress() failed:", repr(e2))
        print("Traceback:")
        traceback.print_exc()
        return False

# Pick one representative compressed TS if available (e.g., JPEG2000/JPEG baseline/RLE)
compressed_uids = [
    str(pydicom.uid.JPEG2000Lossless),
    str(pydicom.uid.JPEG2000),
    str(pydicom.uid.JPEGBaseline),
    str(pydicom.uid.JPEGExtended),
    str(pydicom.uid.JPEGLSLossless),
    str(pydicom.uid.JPEGLSNearLossless),
    str(pydicom.uid.RLELossless),
]

def first_existing_sample(samples_dict, preferred_uids):
    # try preferred compressed first, else any
    for uid in preferred_uids:
        if uid in samples_dict:
            return samples_dict[uid]
    # fallback to any available key
    for uid, path in samples_dict.items():
        if uid not in ("<missing>", "<read_error>"):
            return path
    return None

cand_train = first_existing_sample(train_ts_samples, compressed_uids)
cand_test = first_existing_sample(test_ts_samples, compressed_uids)
print("Candidate train file:", cand_train)
print("Candidate test  file:", cand_test)

ok_any = False
if cand_train:
    ok_any |= try_decode(cand_train)
if cand_test and not ok_any:
    ok_any |= try_decode(cand_test)

print("Decode success?", ok_any)
print("pylibjpeg import check:")
try:
    import pylibjpeg, pylibjpeg_openjpeg, pylibjpeg_libjpeg, pylibjpeg_rle  # noqa
    print("pylibjpeg modules present")
except Exception as e:
    print("pylibjpeg import failed:", repr(e))
print("gdcm import check:")
try:
    import gdcm  # from python-gdcm
    print("gdcm imported OK")
except Exception as e:
    print("gdcm import failed:", repr(e))
print("=== End DICOM decompression debug ===", flush=True)

=== DICOM decompression debug ===


Handlers registered:
 - pydicom.pixel_data_handlers.numpy_handler
 - pydicom.pixel_data_handlers.gdcm_handler
 - pydicom.pixel_data_handlers.pillow_handler
 - pydicom.pixel_data_handlers.jpeg_ls_handler
 - pydicom.pixel_data_handlers.pylibjpeg_handler
 - pydicom.pixel_data_handlers.rle_handler


Train TS counts (sampled): {'1.2.840.10008.1.2.1': 1864, '1.2.840.10008.1.2.4.70': 136}
Test  TS counts (sampled): {'1.2.840.10008.1.2.1': 592, '1.2.840.10008.1.2.4.70': 46}
Candidate train file: train/14204d559d6c/2603e1c933cf/061067e90779.dcm
Candidate test  file: test/30e45593ba08/c5c0a57e6e4c/004cbd797cd1.dcm

-- Try decode: train/14204d559d6c/2603e1c933cf/061067e90779.dcm
TransferSyntaxUID: 1.2.840.10008.1.2.1 Explicit VR Little Endian
PhotometricInterpretation: MONOCHROME2
Rows x Cols: 2801 2798
pixel_array OK: (2801, 2798) uint16
Decode success? True
pylibjpeg import check:
pylibjpeg import failed: ModuleNotFoundError("No module named 'pylibjpeg'")
gdcm import check:
gdcm imported OK
=== End DICOM decompression debug ===


In [12]:
import sys, subprocess
from importlib import import_module

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

print("=== Install/verify pylibjpeg stack ===", flush=True)
pkgs = [
    ("pylibjpeg", "2.0.2"),
    ("pylibjpeg-libjpeg", "2.1.0"),
    ("pylibjpeg-openjpeg", "2.3.0"),
    ("pylibjpeg-rle", "2.0.0"),
]
for name, ver in pkgs:
    try:
        import_module(name)
        print(f"{name} already importable")
    except Exception:
        pip("install", f"{name}=={ver}")

print("Re-check imports:")
for name, _ in pkgs:
    try:
        import_module(name)
        print(f"OK: {name}")
    except Exception as e:
        print(f"FAIL: {name}", repr(e))

print("pydicom handlers after install:", flush=True)
import pydicom
from pydicom import config as pydicom_config
for h in pydicom_config.pixel_data_handlers:
    print(" -", getattr(h, "__name__", str(h)))
print("=== Done ===", flush=True)

=== Install/verify pylibjpeg stack ===


> install pylibjpeg==2.0.2


ERROR: Could not find a version that satisfies the requirement pylibjpeg==2.0.2 (from versions: 1.0.0, 1.1.0, 1.1.1, 1.2.0, 1.3.0, 1.4.0, 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for pylibjpeg==2.0.2


CalledProcessError: Command '['/usr/bin/python3.11', '-m', 'pip', 'install', 'pylibjpeg==2.0.2']' returned non-zero exit status 1.

In [13]:
import sys, subprocess
from importlib import import_module
from pathlib import Path
import pydicom
from pydicom.uid import UID
from pydicom import config as pydicom_config

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

print("=== Install pylibjpeg plugins (no strict pins) + verify ===", flush=True)
to_install = ["pylibjpeg", "pylibjpeg-libjpeg", "pylibjpeg-openjpeg", "pylibjpeg-rle"]
for pkg in to_install:
    try:
        import_module(pkg)
        print(f"OK already: {pkg}")
    except Exception:
        pip("install", pkg)

print("Handlers now registered:")
for h in pydicom_config.pixel_data_handlers:
    print(" -", getattr(h, "__name__", str(h)))

print("=== Try decoding a JPEG Lossless (Process 14) sample (1.2.840.10008.1.2.4.70) ===")
def find_ts_sample(root: str, target_ts: str):
    for p in Path(root).rglob("*.dcm"):
        try:
            ds = pydicom.dcmread(str(p), stop_before_pixels=True, force=True)
            ts = str(getattr(ds.file_meta, "TransferSyntaxUID", ""))
            if ts == target_ts:
                return str(p)
        except Exception:
            pass
    return None

TS_JPEG_LOSSLESS_14 = "1.2.840.10008.1.2.4.70"
pth = find_ts_sample("train", TS_JPEG_LOSSLESS_14) or find_ts_sample("test", TS_JPEG_LOSSLESS_14)
print("Sample for 1.2.840.10008.1.2.4.70:", pth)
ok = False
if pth:
    ds = pydicom.dcmread(pth, force=True)
    print("TransferSyntaxUID:", ds.file_meta.TransferSyntaxUID, UID(ds.file_meta.TransferSyntaxUID).name)
    try:
        arr = ds.pixel_array
        print("pixel_array OK:", getattr(arr, 'shape', None), getattr(arr, 'dtype', None))
        ok = True
    except Exception as e:
        print("pixel_array failed:", repr(e))
        # Try explicit gdcm path as fallback
        try:
            import gdcm, numpy as np
            r = gdcm.ImageReader()
            r.SetFileName(pth)
            if r.Read():
                img = r.GetImage()
                pi = img.GetPixelFormat()
                dims = img.GetDimension(0), img.GetDimension(1)
                buf = img.GetBuffer()
                # Assume 16-bit mono
                arr2 = np.frombuffer(buf, dtype=np.uint16).reshape(dims[::-1])
                print("GDCM fallback OK:", arr2.shape, arr2.dtype)
                ok = True
            else:
                print("GDCM ImageReader failed to read file")
        except Exception as e2:
            print("GDCM fallback failed:", repr(e2))
print("Decode JPEG Lossless success?", ok, flush=True)
print("=== End install/test ===", flush=True)

=== Install pylibjpeg plugins (no strict pins) + verify ===


> install pylibjpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.1 MB/s eta 0:00:00


> install pylibjpeg-libjpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 23.3 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.8/272.8 KB 12.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.7/266.7 KB 452.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 154.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 108.3 MB/s eta 0:00:00


Handlers now registered:
 - pydicom.pixel_data_handlers.numpy_handler
 - pydicom.pixel_data_handlers.gdcm_handler
 - pydicom.pixel_data_handlers.pillow_handler
 - pydicom.pixel_data_handlers.jpeg_ls_handler
 - pydicom.pixel_data_handlers.pylibjpeg_handler
 - pydicom.pixel_data_handlers.rle_handler
=== Try decoding a JPEG Lossless (Process 14) sample (1.2.840.10008.1.2.4.70) ===
Sample for 1.2.840.10008.1.2.4.70: train/b42cb3072150/b561ebcb92fb/09c6d5c914e2.dcm
TransferSyntaxUID: 1.2.840.10008.1.2.4.70 JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
pixel_array failed: RuntimeError('The following handlers are available to decode the pixel data however they are missing required dependencies: GDCM (req. ), pylibjpeg (req. )')


GDCM fallback failed: TypeError("a bytes-like object is required, not 'str'")
Decode JPEG Lossless success? False


=== End install/test ===


In [19]:
import os
os.environ.setdefault("OMP_NUM_THREADS", "1")

from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import pydicom, gdcm, tempfile, uuid
from concurrent.futures import ProcessPoolExecutor, as_completed
from pydicom.uid import ExplicitVRLittleEndian, ImplicitVRLittleEndian
from pydicom.pixel_data_handlers import numpy_handler, gdcm_handler

print("=== DICOM -> PNG cache (ProcessPool + GDCM decompress-to-temp for compressed) ===", flush=True)
cache_root = Path("cache")
cache_train = cache_root/"train"
cache_test = cache_root/"test"
cache_tmp = cache_root/"tmp"
cache_root.mkdir(parents=True, exist_ok=True)
cache_train.mkdir(parents=True, exist_ok=True)
cache_test.mkdir(parents=True, exist_ok=True)
cache_tmp.mkdir(parents=True, exist_ok=True)

UNCOMPRESSED = {str(ImplicitVRLittleEndian), str(ExplicitVRLittleEndian)}

def worker_init():
    import pydicom, gdcm
    from pydicom.pixel_data_handlers import numpy_handler
    # Pin to numpy-only so pydicom never attempts compressed handlers in workers
    pydicom.config.pixel_data_handlers = [numpy_handler]
    gdcm.ImageReader()  # warm-up

def gdcm_decompress_to_file(src: str, dst: str):
    r = gdcm.ImageReader()
    r.SetFileName(src)
    if not r.Read():
        raise RuntimeError("GDCM read failed")
    change = gdcm.ImageChangeTransferSyntax()
    change.SetInput(r.GetImage())
    change.SetTransferSyntax(gdcm.TransferSyntax(gdcm.TransferSyntax.ExplicitVRLittleEndian))
    if not change.Change():
        raise RuntimeError("GDCM change transfer syntax failed")
    w = gdcm.ImageWriter()
    w.SetFile(r.GetFile())
    w.SetImage(change.GetOutput())
    w.SetFileName(dst)
    if not w.Write():
        raise RuntimeError("GDCM write failed")

def load_array(path: str):
    ds_hdr = pydicom.dcmread(path, stop_before_pixels=True, force=True)
    ts = str(getattr(ds_hdr.file_meta, "TransferSyntaxUID", ""))
    slope = float(getattr(ds_hdr, "RescaleSlope", 1.0))
    inter = float(getattr(ds_hdr, "RescaleIntercept", 0.0))
    invert = getattr(ds_hdr, "PhotometricInterpretation", "") == "MONOCHROME1"
    if ts in UNCOMPRESSED:
        ds = pydicom.dcmread(path, force=True)
        arr = ds.pixel_array
    else:
        # Decompress with GDCM to a temp uncompressed DICOM, then read via pydicom+numpy
        tmp_path = str(cache_tmp / f"{Path(path).stem}_{uuid.uuid4().hex}.dcm")
        gdcm_decompress_to_file(path, tmp_path)
        try:
            ds = pydicom.dcmread(tmp_path, force=True)
            arr = ds.pixel_array
        finally:
            try:
                os.remove(tmp_path)
            except Exception:
                pass
    arr = arr.astype(np.float32) * slope + inter
    if invert:
        arr = -arr  # invert after rescale
    return arr, int(getattr(ds_hdr, 'Rows', 0) or 0), int(getattr(ds_hdr, 'Columns', 0) or 0)

def to_uint8_window(arr: np.ndarray, p_low=2, p_high=98):
    lo = float(np.percentile(arr, p_low))
    hi = float(np.percentile(arr, p_high))
    if hi <= lo:
        hi = lo + 1.0
    x = np.clip((arr - lo) / (hi - lo), 0, 1)
    return (x * 255.0).astype(np.uint8)

def build_stem_map(root: Path):
    stem2path = {}
    for p in root.rglob("*.dcm"):
        stem2path[p.stem] = str(p)
    return stem2path

print("Indexing DICOM paths...", flush=True)
train_map = build_stem_map(Path("train"))
test_map = build_stem_map(Path("test"))
print(f"train_map: {len(train_map)} stems, test_map: {len(test_map)} stems", flush=True)

train_img_df = pd.read_csv("train_image_level.csv")
train_img_df["image_id"] = train_img_df["id"].str.replace("_image", "", regex=False)
sub = pd.read_csv("sample_submission.csv")
test_img_ids = sub.loc[sub["id"].str.endswith("_image"), "id"].str.replace("_image", "", regex=False).tolist()

train_stems = [s for s in train_img_df["image_id"].tolist() if s in train_map]
test_stems = [s for s in test_img_ids if s in test_map]
print(f"To cache: train {len(train_stems)}, test {len(test_stems)}", flush=True)

def process_one(stem: str, is_train: bool):
    mp = train_map if is_train else test_map
    out_dir = cache_train if is_train else cache_test
    src = mp.get(stem)
    if src is None:
        return None
    dst = str(out_dir / f"{stem}.png")
    if Path(dst).exists():
        ds_hdr = pydicom.dcmread(src, stop_before_pixels=True, force=True)
        h = int(getattr(ds_hdr, 'Rows', 0) or 0)
        w = int(getattr(ds_hdr, 'Columns', 0) or 0)
        return (stem, h, w, dst)
    arr, h, w = load_array(src)
    img = to_uint8_window(arr, 2, 98)
    ok = cv2.imwrite(dst, img)
    if not ok:
        raise RuntimeError("cv2.imwrite failed")
    return (stem, h, w, dst)

def cache_many(pairs, workers=6):
    out = []
    total = len(pairs)
    with ProcessPoolExecutor(max_workers=workers, initializer=worker_init) as ex:
        futs = {ex.submit(process_one, stem, is_train): (stem, is_train) for stem, is_train in pairs}
        for i, f in enumerate(as_completed(futs), 1):
            stem, _ = futs[f]
            try:
                r = f.result(),
                r = r[0] if isinstance(r, tuple) and len(r) == 1 else r
                if r is not None:
                    out.append(r)
            except Exception as e:
                print("Err:", stem, e, flush=True)
            if i % 200 == 0 or i == total:
                print(f"Processed {i}/{total}", flush=True)
    return out

pairs = [(s, True) for s in train_stems] + [(s, False) for s in test_stems]
res = cache_many(pairs, workers=6)
meta = pd.DataFrame(res, columns=["image_id","orig_h","orig_w","png_path"])
meta_path = cache_root/"meta_image_shapes.csv"
meta.to_csv(meta_path, index=False)
print("Cached PNGs and wrote:", meta_path, "rows=", len(meta), flush=True)
print(meta.head(), flush=True)
print("Cache complete.", flush=True)

=== DICOM -> PNG cache (ProcessPool + GDCM decompress-to-temp for compressed) ===


Indexing DICOM paths...


train_map: 5696 stems, test_map: 638 stems


To cache: train 5696, test 638


Err: 0452c373f62b A process in the process pool was terminated abruptly while the future was running or pending.


...

Err: ffe942c8655f A process in the process pool was terminated abruptly while the future was running or pending.


Processed 6334/6334


Cached PNGs and wrote: cache/meta_image_shapes.csv rows= 194


       image_id  orig_h  orig_w                      png_path
0  0841677e7c36    2330    2846  cache/train/0841677e7c36.png
1  086f1ab39050    2320    2832  cache/train/086f1ab39050.png
2  000c3a3f293f    2320    2832  cache/train/000c3a3f293f.png
3  085d6edba7cb    2290    2800  cache/train/085d6edba7cb.png
4  02366ec75168    2336    2836  cache/train/02366ec75168.png


Cache complete.


In [17]:
import sys, subprocess
from pathlib import Path
import pydicom
from pydicom.uid import UID

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

print("=== Install SimpleITK and smoke test JPEG Lossless decode ===", flush=True)
pip("install", "SimpleITK==2.3.1")
import SimpleITK as sitk

def find_ts_sample(root: str, target_ts: str):
    for p in Path(root).rglob("*.dcm"):
        try:
            ds = pydicom.dcmread(str(p), stop_before_pixels=True, force=True)
            ts = str(getattr(ds.file_meta, "TransferSyntaxUID", ""))
            if ts == target_ts:
                return str(p)
        except Exception:
            pass
    return None

TS_JPEG_LOSSLESS_14 = "1.2.840.10008.1.2.4.70"
pth = find_ts_sample("train", TS_JPEG_LOSSLESS_14) or find_ts_sample("test", TS_JPEG_LOSSLESS_14)
print("Sample JPEG Lossless path:", pth)
ok = False
if pth:
    try:
        img = sitk.ReadImage(pth)
        arr = sitk.GetArrayFromImage(img)  # shape: [z,y,x], expect z=1
        print("SimpleITK read OK:", arr.shape, arr.dtype)
        ok = True
    except Exception as e:
        print("SimpleITK read failed:", e)
print("SITK JPEG Lossless decode success?", ok, flush=True)

In [22]:
import os
from pathlib import Path
import uuid
import numpy as np
import pandas as pd
import cv2
import pydicom
from pydicom.uid import ExplicitVRLittleEndian, ImplicitVRLittleEndian
from pydicom.pixel_data_handlers import numpy_handler as pyd_np_handler

# Single-threaded safety for codecs
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPJ_NUM_THREADS", "1")
os.environ.setdefault("GDCM_MAX_THREAD_COUNT", "1")

print("=== Sequential DICOM -> PNG cache (no parallelism) [UNCOMPRESSED-ONLY PILOT] ===", flush=True)
cache_root = Path("cache")
cache_train = cache_root/"train"
cache_test = cache_root/"test"
cache_tmp = cache_root/"tmp_seq"
cache_root.mkdir(parents=True, exist_ok=True)
cache_train.mkdir(parents=True, exist_ok=True)
cache_test.mkdir(parents=True, exist_ok=True)
cache_tmp.mkdir(parents=True, exist_ok=True)

UNCOMPRESSED = {str(ImplicitVRLittleEndian), str(ExplicitVRLittleEndian)}

# Force only numpy handler to avoid compressed paths inside pydicom
pydicom.config.pixel_data_handlers = [pyd_np_handler]

def build_stem_map(root: Path):
    stem2path = {}
    for p in root.rglob("*.dcm"):
        stem2path[p.stem] = str(p)
    return stem2path

def load_array_uncompressed(path: str):
    ds = pydicom.dcmread(path, force=True)
    slope = float(getattr(ds, "RescaleSlope", 1.0))
    inter = float(getattr(ds, "RescaleIntercept", 0.0))
    invert = getattr(ds, "PhotometricInterpretation", "") == "MONOCHROME1"
    arr = ds.pixel_array.astype(np.float32) * slope + inter
    if invert:
        arr = -arr
    h = int(getattr(ds, 'Rows', 0) or 0)
    w = int(getattr(ds, 'Columns', 0) or 0)
    return arr, h, w

def to_uint8_window(arr: np.ndarray, p_low=2, p_high=98):
    lo = float(np.percentile(arr, p_low))
    hi = float(np.percentile(arr, p_high))
    if hi <= lo:
        hi = lo + 1.0
    x = np.clip((arr - lo) / (hi - lo), 0, 1)
    return (x * 255.0).astype(np.uint8)

print("Indexing DICOM paths...", flush=True)
train_map = build_stem_map(Path("train"))
test_map = build_stem_map(Path("test"))
print(f"train_map: {len(train_map)} stems, test_map: {len(test_map)} stems", flush=True)

train_img_df = pd.read_csv("train_image_level.csv")
train_img_df["image_id"] = train_img_df["id"].str.replace("_image", "", regex=False)
sub = pd.read_csv("sample_submission.csv")
test_img_ids = sub.loc[sub["id"].str.endswith("_image"), "id"].str.replace("_image", "", regex=False).tolist()

train_stems = [s for s in train_img_df["image_id"].tolist() if s in train_map]
test_stems = [s for s in test_img_ids if s in test_map]
pairs = [(s, True) for s in train_stems] + [(s, False) for s in test_stems]
print(f"To cache sequentially (pilot uncompressed-only): {len(pairs)} images", flush=True)

results = []
errors = 0
skipped_compressed = 0
for i, (stem, is_train) in enumerate(pairs, 1):
    mp = train_map if is_train else test_map
    out_dir = cache_train if is_train else cache_test
    src = mp.get(stem)
    if src is None:
        continue
    dst = str(out_dir / f"{stem}.png")
    # Read header to determine transfer syntax
    try:
        ds_hdr = pydicom.dcmread(src, stop_before_pixels=True, force=True)
        ts = str(getattr(ds_hdr.file_meta, "TransferSyntaxUID", ""))
        h = int(getattr(ds_hdr, 'Rows', 0) or 0)
        w = int(getattr(ds_hdr, 'Columns', 0) or 0)
    except Exception:
        ts = ""
        h = w = 0
    if ts not in UNCOMPRESSED:
        skipped_compressed += 1
        if i % 200 == 0:
            print(f"Skip compressed so far: {skipped_compressed}", flush=True)
        continue
    if os.path.exists(dst):
        results.append((stem, h, w, dst))
        continue
    try:
        arr, h, w = load_array_uncompressed(src)
        img = to_uint8_window(arr, 2, 98)
        ok = cv2.imwrite(dst, img)
        if ok:
            results.append((stem, h, w, dst))
        else:
            errors += 1
    except Exception as e:
        errors += 1
        if errors <= 20:
            print(f"Err [{stem}]: {e}", flush=True)
    if i % 200 == 0 or i == len(pairs):
        print(f"Progress: {i}/{len(pairs)} (saved={len(results)}, errors={errors}, skipped_compressed={skipped_compressed})", flush=True)

meta = pd.DataFrame(results, columns=["image_id","orig_h","orig_w","png_path"])
meta_path = cache_root/"meta_image_shapes_seq.csv"
meta.to_csv(meta_path, index=False)
print("Sequential cache (uncompressed-only) done. Wrote:", meta_path, "rows=", len(meta), "errors=", errors, "skipped_compressed=", skipped_compressed, flush=True)

=== Sequential DICOM -> PNG cache (no parallelism) [UNCOMPRESSED-ONLY PILOT] ===


Indexing DICOM paths...


train_map: 5696 stems, test_map: 638 stems


To cache sequentially (pilot uncompressed-only): 6334 images


Skip compressed so far: 6


Skip compressed so far: 52


Skip compressed so far: 319


Sequential cache (uncompressed-only) done. Wrote: cache/meta_image_shapes_seq.csv rows= 5894 errors= 0 skipped_compressed= 440


In [23]:
import os, shutil, json, math, ast
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold

print("=== Prepare YOLOv8 dataset from cached PNGs (cached-uncompressed subset) ===", flush=True)
cache_meta = Path("cache/meta_image_shapes_seq.csv")
assert cache_meta.exists(), "Run caching first to create cache/meta_image_shapes_seq.csv"
meta = pd.read_csv(cache_meta)

img_df = pd.read_csv("train_image_level.csv")
img_df["image_id"] = img_df["id"].str.replace("_image", "", regex=False)
studies = img_df[["image_id","StudyInstanceUID"]].drop_duplicates()
df = img_df.merge(meta, on="image_id", how="inner")
print("Train images available (cached):", len(df))

out_root = Path("yolo_data")
img_root = out_root / "images"
lbl_root = out_root / "labels"
for d in [out_root, img_root, lbl_root]:
    d.mkdir(parents=True, exist_ok=True)

def yolo_label_path(image_png_path: str):
    stem = Path(image_png_path).stem
    return lbl_root / f"{stem}.txt"

def write_yolo_label(row):
    # Create/overwrite label file for this image
    lp = yolo_label_path(row["png_path"])
    h = float(row["orig_h"])
    w = float(row["orig_w"])
    boxes_str = row.get("boxes", np.nan)
    lines = []
    if isinstance(boxes_str, str) and boxes_str.strip().startswith("["):
        try:
            boxes = ast.literal_eval(boxes_str)
            for b in boxes:
                x, y, bw, bh = float(b["x"]), float(b["y"]), float(b["width"]), float(b["height"])
                cx = (x + bw/2.0) / w
                cy = (y + bh/2.0) / h
                nw = bw / w
                nh = bh / h
                # class 0 only (opacity)
                cx = min(max(cx, 0.0), 1.0)
                cy = min(max(cy, 0.0), 1.0)
                nw = min(max(nw, 0.0), 1.0)
                nh = min(max(nh, 0.0), 1.0)
                lines.append(f"0 {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")
        except Exception as e:
            lines = []
    # If no boxes, ensure an empty file exists
    lp.parent.mkdir(parents=True, exist_ok=True)
    with open(lp, "w") as f:
        if lines:
            f.write("\n".join(lines))

# Write labels
for i, row in df.iterrows():
    write_yolo_label(row)
    if (i+1) % 1000 == 0 or i+1 == len(df):
        print(f"Labels written: {i+1}/{len(df)}", flush=True)

# Build 3-fold GroupKFold by StudyInstanceUID on available cached images
folds = 3
gkf = GroupKFold(n_splits=folds)
groups = df["StudyInstanceUID"].values
indices = np.arange(len(df))

fold_dirs = []
for fold, (tr_idx, va_idx) in enumerate(gkf.split(indices, groups=groups), 1):
    tr_df = df.iloc[tr_idx].reset_index(drop=True)
    va_df = df.iloc[va_idx].reset_index(drop=True)
    fold_img_train = img_root / f"fold{fold}" / "train"
    fold_img_val = img_root / f"fold{fold}" / "val"
    for d in [fold_img_train, fold_img_val]:
        d.mkdir(parents=True, exist_ok=True)
    # Symlink images to fold directories
    def link_images(sub_df, dst_dir):
        created = 0
        for p in sub_df["png_path"].tolist():
            src = Path(p).resolve()
            dst = dst_dir / Path(p).name
            if not dst.exists():
                try:
                    os.symlink(src, dst)
                except FileExistsError:
                    pass
                except OSError:
                    # fallback copy if symlink not permitted
                    shutil.copy2(src, dst)
                created += 1
        return created
    n_tr = link_images(tr_df, fold_img_train)
    n_va = link_images(va_df, fold_img_val)
    # Write data.yaml for this fold
    data_yaml = out_root / f"data_fold{fold}.yaml"
    data_yaml.write_text(
        f"path: {out_root.resolve()}\n"
        f"train: {fold_img_train.resolve()}\n"
        f"val: {fold_img_val.resolve()}\n"
        f"names: ['opacity']\n"
        f"nc: 1\n"
    )
    fold_dirs.append((fold, str(data_yaml)))
    print(f"Fold {fold}: train={len(tr_df)} (linked {n_tr}), val={len(va_df)} (linked {n_va}) -> {data_yaml}", flush=True)

print("YOLO dataset prepared. Folds and yaml:")
for f, y in fold_dirs:
    print(f" - fold {f}: {y}")

print("Next: install ultralytics and train fold1 with imgsz=896 or 1024.")

=== Prepare YOLOv8 dataset from cached PNGs (cached-uncompressed subset) ===


Train images available (cached): 5302
Labels written: 1000/5302


Labels written: 2000/5302


Labels written: 3000/5302


Labels written: 4000/5302


...

Labels written: 5302/5302


Fold 1: train=3534 (linked 3534), val=1768 (linked 1768) -> yolo_data/data_fold1.yaml


Fold 2: train=3535 (linked 3535), val=1767 (linked 1767) -> yolo_data/data_fold2.yaml


Fold 3: train=3535 (linked 3535), val=1767 (linked 1767) -> yolo_data/data_fold3.yaml


YOLO dataset prepared. Folds and yaml:
 - fold 1: yolo_data/data_fold1.yaml
 - fold 2: yolo_data/data_fold2.yaml
 - fold 3: yolo_data/data_fold3.yaml
Next: install ultralytics and train fold1 with imgsz=896 or 1024.


In [24]:
from pathlib import Path
import os, shutil
import pandas as pd

print("=== Mirror YOLO label files into fold-specific directories ===", flush=True)
out_root = Path("yolo_data")
img_root = out_root / "images"
lbl_root = out_root / "labels"
assert img_root.exists() and lbl_root.exists(), "Run the YOLO prep cell first"

def ensure_symlink(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if dst.exists():
        return False
    try:
        os.symlink(src.resolve(), dst)
    except FileExistsError:
        pass
    except OSError:
        shutil.copy2(src, dst)
    return True

total_linked = 0
for fold in (1,2,3):
    for split in ("train","val"):
        img_dir = img_root / f"fold{fold}" / split
        lbl_dir = lbl_root / f"fold{fold}" / split
        created = 0
        for img_path in img_dir.glob("*.png"):
            stem = img_path.stem
            src_lbl = lbl_root / f"{stem}.txt"
            if not src_lbl.exists():
                # create an empty label file if missing
                src_lbl.parent.mkdir(parents=True, exist_ok=True)
                src_lbl.write_text("")
            dst_lbl = lbl_dir / src_lbl.name
            if ensure_symlink(src_lbl, dst_lbl):
                created += 1
        total_linked += created
        print(f"Fold {fold} {split}: linked {created} labels -> {lbl_dir}", flush=True)

print("Label mirroring complete. Total linked/copied:", total_linked, flush=True)
print("Next: install ultralytics and train fold 1 (imgsz=896/1024).", flush=True)

=== Mirror YOLO label files into fold-specific directories ===


Fold 1 train: linked 3534 labels -> yolo_data/labels/fold1/train


Fold 1 val: linked 1768 labels -> yolo_data/labels/fold1/val


Fold 2 train: linked 3535 labels -> yolo_data/labels/fold2/train


Fold 2 val: linked 1767 labels -> yolo_data/labels/fold2/val


Fold 3 train: linked 3535 labels -> yolo_data/labels/fold3/train


Fold 3 val: linked 1767 labels -> yolo_data/labels/fold3/val


Label mirroring complete. Total linked/copied: 15906


Next: install ultralytics and train fold 1 (imgsz=896/1024).


In [25]:
import sys, subprocess, os

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

print("=== Install Ultralytics (YOLOv8) ===", flush=True)
# Ensure torch stack remains fixed via constraints
pip("install", "-c", "constraints.txt", "ultralytics==8.2.103", "--upgrade-strategy", "only-if-needed")
import ultralytics
print("Ultralytics version:", ultralytics.__version__, flush=True)
print("Install complete.")

=== Install Ultralytics (YOLOv8) ===


> install -c constraints.txt ultralytics==8.2.103 --upgrade-strategy only-if-needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.1/875.1 KB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 438.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 KB 77.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 175.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.2/291.2 KB 534.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 170.7 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 197.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 509.1 MB/s eta 0:00:00


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/app/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics version: 8.2.103


Install complete.


In [ ]:
import os, time
from pathlib import Path
print("=== YOLOv8s train: fold1 ===", flush=True)
yaml_path = Path("yolo_data/data_fold1.yaml").resolve()
assert yaml_path.exists(), f"Missing {yaml_path}"
from ultralytics import YOLO
model = YOLO("yolov8s.pt")  # COCO-pretrained
t0 = time.time()
results = model.train(
    data=str(yaml_path),
    imgsz=896,
    epochs=20,
    batch=16,
    device=0,
    workers=4,
    project="runs_yolo",
    name="fold1_s896_e20",
    exist_ok=True,
    seed=42,
    patience=3,
    # conservative augments
    mosaic=0.0, mixup=0.0,
    hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
    fliplr=0.5, flipud=0.0,
    degrees=5.0, translate=0.05, scale=0.1, shear=0.0,
)
print("Train done in", round(time.time()-t0,1), "s", flush=True)
print("Best model:", results.save_dir, flush=True)

=== YOLOv8s train: fold1 ===


  0%|          | 0.00/21.5M [00:00<?, ?B/s]

 47%|████▋     | 10.1M/21.5M [00:00<00:00, 49.5MB/s]

 69%|██████▉   | 14.9M/21.5M [00:00<00:00, 24.1MB/s]

 92%|█████████▏| 19.8M/21.5M [00:00<00:00, 29.9MB/s]

...

train: Scanning /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-015252/yolo_data/labels/fold1/train...:   0%|          | 0/3534 [00:00<?, ?it/s]

train: Scanning /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-015252/yolo_data/labels/fold1/train... 1 images, 1 backgrounds, 0 corrupt:   0%|          | 1/3534 [00:00<10:56,  5.38it/s]

train: Scanning /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-015252/yolo_data/labels/fold1/train... 3 images, 1 backgrounds, 0 corrupt:   0%|          | 3/3534 [00:00<06:29,  9.07it/s]

train: Scanning /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-015252/yolo_data/labels/fold1/train... 14 images, 5 backgrounds, 0 corrupt:   0%|          | 14/3534 [00:00<01:24, 41.58it/s]

train: Scanning /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-015252/yolo_data/labels/fold1/train... 20 images, 5 backgrounds, 0 corrupt:   1%|          | 20/3534 [00:00<01:33, 37.67it/s]

In [ ]:
import os, sys, json, textwrap, subprocess, time
from pathlib import Path
import pandas as pd

print("=== Launch compressed DICOM caching in subprocess shards (sequential per shard) ===", flush=True)
# Env safety for each subprocess
ENV_KV = {
    "OMP_NUM_THREADS": "1",
    "OPJ_NUM_THREADS": "1",
    "GDCM_MAX_THREAD_COUNT": "1",
}

cache_root = Path("cache")
cache_train = cache_root/"train"
cache_test = cache_root/"test"
cache_root.mkdir(parents=True, exist_ok=True)
cache_train.mkdir(parents=True, exist_ok=True)
cache_test.mkdir(parents=True, exist_ok=True)

# 1) Determine missing stems (not cached in uncompressed-only pass)
meta_seq = pd.read_csv(cache_root/"meta_image_shapes_seq.csv")
cached_set = set(meta_seq["image_id"].astype(str))
img_df = pd.read_csv("train_image_level.csv")
img_df["image_id"] = img_df["id"].str.replace("_image", "", regex=False)
train_ids = set(img_df["image_id"].astype(str))
sub = pd.read_csv("sample_submission.csv")
test_ids = set(sub.loc[sub["id"].str.endswith("_image"), "id"].str.replace("_image", "", regex=False).astype(str))

# Build stem->path maps
def build_stem_map(root: Path):
    d = {}
    for p in root.rglob("*.dcm"):
        d[p.stem] = str(p)
    return d
train_map = build_stem_map(Path("train"))
test_map = build_stem_map(Path("test"))

all_needed = [(s, True) for s in train_ids if s in train_map] + [(s, False) for s in test_ids if s in test_map]
missing_pairs = [(s, is_tr) for (s, is_tr) in all_needed if s not in cached_set]
print(f"Missing to cache (likely compressed): {len(missing_pairs)}", flush=True)
if not missing_pairs:
    print("Nothing to do.", flush=True)
    raise SystemExit

# Optional: group by StudyInstanceUID for train; test has no study uid in CSV, so keep as-is
study_map = img_df.set_index("image_id")["StudyInstanceUID"].to_dict()
def group_key(pair):
    s, is_tr = pair
    return (study_map.get(s, s) if is_tr else f"test_{s}")

groups = {}
for pair in missing_pairs:
    k = group_key(pair)
    groups.setdefault(k, []).append(pair)

# Shard groups into N roughly-equal shards
N = min(4, max(2, os.cpu_count() // 16))  # 2-4 shards
shards = [[] for _ in range(N)]
for i, (_, glist) in enumerate(groups.items()):
    shards[i % N].extend(glist)
for i in range(N):
    print(f"Shard {i}: {len(shards[i])} pairs", flush=True)

# 2) Write shard lists and a small worker script to disk
tools_dir = Path("tools")
tools_dir.mkdir(parents=True, exist_ok=True)
worker_py = tools_dir/"cache_shard_seq.py"
worker_src = """
import os, sys, uuid, json
from pathlib import Path
import numpy as np
import cv2
import pydicom
from pydicom.uid import ExplicitVRLittleEndian, ImplicitVRLittleEndian

os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPJ_NUM_THREADS", "1")
os.environ.setdefault("GDCM_MAX_THREAD_COUNT", "1")

UNCOMP = {str(ImplicitVRLittleEndian), str(ExplicitVRLittleEndian)}

def build_map(root: Path):
    d = {}
    for p in root.rglob("*.dcm"):
        d[p.stem] = str(p)
    return d

def to_uint8_window(arr, p_low=2, p_high=98):
    lo = float(np.percentile(arr, p_low))
    hi = float(np.percentile(arr, p_high))
    if hi <= lo:
        hi = lo + 1.0
    x = np.clip((arr - lo) / (hi - lo), 0, 1)
    return (x * 255.0).astype(np.uint8)

def gdcm_decompress_to_file(src: str, dst: str):
    import gdcm
    r = gdcm.ImageReader()
    r.SetFileName(src)
    if not r.Read():
        raise RuntimeError("GDCM read failed")
    change = gdcm.ImageChangeTransferSyntax()
    change.SetInput(r.GetImage())
    change.SetTransferSyntax(gdcm.TransferSyntax(gdcm.TransferSyntax.ExplicitVRLittleEndian))
    if not change.Change():
        raise RuntimeError("GDCM change transfer syntax failed")
    w = gdcm.ImageWriter()
    w.SetFile(r.GetFile())
    w.SetImage(change.GetOutput())
    w.SetFileName(dst)
    if not w.Write():
        raise RuntimeError("GDCM write failed")

def load_arr(path: str):
    ds_hdr = pydicom.dcmread(path, stop_before_pixels=True, force=True)
    ts = str(getattr(ds_hdr.file_meta, "TransferSyntaxUID", ""))
    slope = float(getattr(ds_hdr, "RescaleSlope", 1.0))
    inter = float(getattr(ds_hdr, "RescaleIntercept", 0.0))
    invert = getattr(ds_hdr, "PhotometricInterpretation", "") == "MONOCHROME1"
    if ts in UNCOMP:
        ds = pydicom.dcmread(path, force=True)
        arr = ds.pixel_array
    else:
        tmp = str(Path(dst_dir)/f"tmp_{Path(path).stem}_{uuid.uuid4().hex}.dcm")
        gdcm_decompress_to_file(path, tmp)
        try:
            ds = pydicom.dcmread(tmp, force=True)
            arr = ds.pixel_array
        finally:
            try: os.remove(tmp)
            except Exception: pass
    arr = arr.astype(np.float32) * slope + inter
    if invert:
        arr = -arr
    return arr, int(getattr(ds_hdr, 'Rows', 0) or 0), int(getattr(ds_hdr, 'Columns', 0) or 0)

if __name__ == "__main__":
    list_path = Path(sys.argv[1])
    out_root = Path(sys.argv[2])
    train_map = build_map(Path("train"))
    test_map = build_map(Path("test"))
    out_root.mkdir(parents=True, exist_ok=True)
    global dst_dir
    dst_dir = out_root/"tmp"
    dst_dir.mkdir(parents=True, exist_ok=True)
    pairs = []
    with open(list_path) as f:
        for line in f:
            s, is_tr = line.strip().split(",")
            pairs.append((s, is_tr == "1"))
    saved = 0
    errs = 0
    for i, (stem, is_tr) in enumerate(pairs, 1):
        mp = train_map if is_tr else test_map
        src = mp.get(stem)
        if not src:
            continue
        out_dir = out_root/("train" if is_tr else "test")
        out_dir.mkdir(parents=True, exist_ok=True)
        dst = str(out_dir / f"{stem}.png")
        if os.path.exists(dst):
            continue
        try:
            arr, h, w = load_arr(src)
            img = to_uint8_window(arr, 2, 98)
            if not cv2.imwrite(dst, img):
                errs += 1
            else:
                saved += 1
        except Exception as e:
            errs += 1
        if i % 100 == 0 or i == len(pairs):
            print(f"[{list_path.name}] Progress {i}/{len(pairs)} saved={saved} errs={errs}", flush=True)
"""
worker_py.write_text(worker_src)

# 3) Write shard lists
lists_dir = tools_dir/"lists"
lists_dir.mkdir(parents=True, exist_ok=True)
list_paths = []
for i, shard in enumerate(shards):
    lp = lists_dir/f"shard_{i}.txt"
    with open(lp, "w") as f:
        for s, is_tr in shard:
            f.write(f"{s},{1 if is_tr else 0}\n")
    list_paths.append(lp)

# 4) Launch subprocesses
procs = []
for i, lp in enumerate(list_paths):
    cmd = [sys.executable, str(worker_py), str(lp), str(cache_root)]
    env = os.environ.copy()
    env.update(ENV_KV)
    p = subprocess.Popen(cmd, env=env)
    procs.append((i, p))
    print(f"Launched shard {i} PID={p.pid} -> {lp}", flush=True)

print("All shards launched. Monitoring...", flush=True)
while procs:
    alive = []
    for i, p in procs:
        ret = p.poll()
        if ret is None:
            alive.append((i, p))
        else:
            print(f"Shard {i} exited code {ret}", flush=True)
    procs = alive
    if procs:
        time.sleep(10)

print("Compressed caching subprocesses finished.", flush=True)
print("Rebuilding meta CSV...", flush=True)
# 5) Rebuild meta from actual cached PNGs
import pydicom
rows = []
for subdir, is_tr in [(cache_train, True), (cache_test, False)]:
    for png in subdir.glob("*.png"):
        stem = png.stem
        dcm = (Path("train") if is_tr else Path("test"))
        cand = list(dcm.rglob(f"**/{stem}.dcm"))
        h = w = 0
        if cand:
            try:
                ds_hdr = pydicom.dcmread(str(cand[0]), stop_before_pixels=True, force=True)
                h = int(getattr(ds_hdr, 'Rows', 0) or 0)
                w = int(getattr(ds_hdr, 'Columns', 0) or 0)
            except Exception:
                pass
        rows.append((stem, h, w, str(png)))
meta_df = pd.DataFrame(rows, columns=["image_id","orig_h","orig_w","png_path"])
meta_df.to_csv(cache_root/"meta_image_shapes_seq.csv", index=False)
print("Meta updated:", cache_root/"meta_image_shapes_seq.csv", "rows=", len(meta_df), flush=True)

## Next steps: OOF/test inference, WBF, study-level mapping, submission
- After fold1 training completes, run OOF inference on its val set with low conf to feed WBF.
- Use HFlip TTA; later ensemble across folds.
- Build simple study-level heuristic; optional LR in a later cell.
- Then run test inference and compose submission.

In [ ]:
import os, time, json, math, gc
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from ultralytics import YOLO

print("=== OOF inference utils (fold1) ===", flush=True)
meta = pd.read_csv("cache/meta_image_shapes_seq.csv")
meta = meta.set_index("image_id")["orig_h"].to_frame().join(meta.set_index("image_id")["orig_w"]).reset_index()
meta.columns = ["image_id","orig_h","orig_w"]
meta_dict = {r.image_id:(int(r.orig_h), int(r.orig_w)) for r in meta.itertuples(index=False)}

def run_infer_dir(weights_path: str, img_dir: str, out_parquet: str, conf=0.001, iou=0.6, imgsz=896, tta=True, max_det=300):
    assert Path(img_dir).exists(), f"Missing {img_dir}"
    model = YOLO(weights_path)
    res = model.predict(source=img_dir, imgsz=imgsz, conf=conf, iou=iou, max_det=max_det, device=0, save=False, verbose=False, augment=bool(tta), agnostic_nms=False, classes=None, half=False)
    rows = []
    for r in res:
        p = Path(r.path)
        image_id = p.stem
        h, w = meta_dict.get(image_id, (r.orig_shape[0], r.orig_shape[1]))
        if r.boxes is None or len(r.boxes) == 0:
            rows.append((image_id, -1, 0, 0, 0, 0))
            continue
        b = r.boxes
        xyxy = b.xyxy.cpu().numpy()  # x1,y1,x2,y2 in letterboxed space mapped back by Ultralytics to original image size
        confs = b.conf.cpu().numpy()
        # clamp and convert to xywh in original pixels
        for (x1,y1,x2,y2), c in zip(xyxy, confs):
            x1 = float(max(0, min(w-1, x1)))
            y1 = float(max(0, min(h-1, y1)))
            x2 = float(max(0, min(w-1, x2)))
            y2 = float(max(0, min(h-1, y2)))
            bw = max(0.0, x2 - x1)
            bh = max(0.0, y2 - y1)
            rows.append((image_id, float(c), x1, y1, bw, bh))
    table = pa.Table.from_pandas(pd.DataFrame(rows, columns=["image_id","conf","x","y","w","h"]))
    Path(out_parquet).parent.mkdir(parents=True, exist_ok=True)
    pq.write_table(table, out_parquet)
    print("Wrote:", out_parquet, "rows=", len(rows), flush=True)

def wait_for_weights(path: str, timeout_s=36000, poll=15):
    t0=time.time()
    while time.time()-t0 < timeout_s:
        if Path(path).exists():
            return True
        time.sleep(poll)
    return False

# Plan to run after training finishes:
fold1_dir = Path("runs_yolo/fold1_s896_e20/weights")
best_w = fold1_dir/"best.pt"
val_dir = Path("yolo_data/images/fold1/val")
oof_dir = Path("yolo_oof")
oof_pq = oof_dir/"fold1_val_preds.parquet"
print("Prepared paths:\n - weights:", best_w, "\n - val_dir:", val_dir, "\n - out:", oof_pq, flush=True)
print("When fold1 completes, run: run_infer_dir(str(best_w), str(val_dir), str(oof_pq), conf=0.001, iou=0.6, imgsz=896, tta=True)", flush=True)

In [ ]:
import os, math, json
from pathlib import Path
import numpy as np
import pandas as pd

print("=== WBF + submission scaffolding ===", flush=True)

def load_parquets(paths):
    dfs = []
    for p in paths:
        p = Path(p)
        if p.exists():
            dfs.append(pd.read_parquet(p))
    return dfs

def run_wbf_from_parquets(parquet_paths, iou_thr=0.50, skip_box_thr=0.0):
    try:
        from ensemble_boxes import weighted_boxes_fusion
    except Exception:
        raise RuntimeError("Install ensemble-boxes before running WBF: pip install ensemble-boxes")
    dfs = load_parquets(parquet_paths)
    if not dfs:
        return {}
    df = pd.concat(dfs, ignore_index=True)
    by_img = {}
    for img_id, g in df.groupby("image_id"):
        boxes_list, scores_list, labels_list = [], [], []
        # Each parquet is assumed to be one model; approximate by splitting equally if shapes match
        # Simpler: treat as one detector list (works if single fold) – for multi-fold, pass separate lists per source.
        # Here we use single-list WBF behavior by duplicating list once per source.
        b = g[["x","y","w","h","conf"]].values
        # convert xywh to normalized [0,1] xyxy using original size if present later (we'll convert back after WBF);
        # for WBF, normalization is optional if all sources share same scale; boxes already in original pixels,
        # so we keep as-is and set conf threshold via skip_box_thr.
        boxes = g[["x","y","w","h"]].values
        xyxy = np.stack([boxes[:,0], boxes[:,1], boxes[:,0]+boxes[:,2], boxes[:,1]+boxes[:,3]], axis=1).tolist()
        scores = g["conf"].clip(lower=0.0, upper=1.0).tolist()
        labels = [0]*len(scores)
        if len(xyxy) == 0:
            by_img[img_id] = ([], [], [])
            continue
        fused_b, fused_s, fused_l = weighted_boxes_fusion([xyxy], [scores], [labels], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        # fused_b are in same pixel scale as input if we didn't normalize; convert to xywh
        fused_b = np.array(fused_b, dtype=float)
        xywh = np.stack([fused_b[:,0], fused_b[:,1], fused_b[:,2]-fused_b[:,0], fused_b[:,3]-fused_b[:,1]], axis=1).tolist()
        by_img[img_id] = (xywh, fused_s, fused_l)
    return by_img

def format_image_prediction_string(img_id, xywh, scores, final_conf=0.28):
    keep = [(s, x, y, w, h) for (x,y,w,h), s in zip(xywh, scores) if s >= final_conf and w > 0 and h > 0]
    if not keep:
        return "none 1 0 0 1 1"
    parts = []
    for s, x, y, w, h in keep:
        parts += ["opacity", f"{s:.6f}", f"{x:.1f}", f"{y:.1f}", f"{w:.1f}", f"{h:.1f}"]
    return " ".join(parts)

def simple_study_mapping(df_img_preds, thresh=0.30):
    # df_img_preds: columns [image_id, study_id, conf, x,y,w,h] per box (optional).
    # Heuristic: if any image in study has s_max>=thresh or >=2 boxes >=0.25 -> Typical else Negative.
    out = {}
    if df_img_preds.empty:
        return out
    for study, g in df_img_preds.groupby("StudyInstanceUID"):
        s_max = g["conf"].max() if "conf" in g else 0.0
        n25 = (g.get("conf", pd.Series([])) >= 0.25).sum() if "conf" in g else 0
        if (s_max >= thresh) or (n25 >= 2):
            # Typical 1, others tiny eps
            out[study] = "typical 0 0 0 1 1"
        else:
            out[study] = "negative 1 0 0 1 1"
    return out

def compose_submission(image_pred_map, final_conf=0.28):
    sub = pd.read_csv("sample_submission.csv")
    out_rows = []
    for rid in sub["id"].tolist():
        if rid.endswith("_image"):
            img_id = rid.replace("_image", "")
            xywh, scores, _ = image_pred_map.get(img_id, ([], [], []))
            ps = format_image_prediction_string(img_id, xywh, scores, final_conf=final_conf)
        else:
            # placeholder study mapping; will overwrite downstream once we build from detections
            ps = "negative 1 0 0 1 1"
        out_rows.append((rid, ps))
    sub_out = pd.DataFrame(out_rows, columns=["id","PredictionString"])
    return sub_out

print("Scaffold ready. After generating fold OOF/test parquet(s), call run_wbf_from_parquets([...]) then compose_submission(...).", flush=True)

In [ ]:
import time
from pathlib import Path
from ultralytics import YOLO

print("=== Training script scaffold: YOLOv8s 3-fold @896 (patience=7, cos_lr, warmup=2) ===", flush=True)
fold_yaml_paths = [
    Path("yolo_data/data_fold1.yaml"),
    Path("yolo_data/data_fold2.yaml"),
    Path("yolo_data/data_fold3.yaml"),
]
for p in fold_yaml_paths:
    assert p.exists(), f"Missing {p}"

def train_fold(yaml_path: Path, fold_name: str):
    model = YOLO("yolov8s.pt")
    t0 = time.time()
    print(f"-- Train {fold_name} --", flush=True)
    res = model.train(
        data=str(yaml_path),
        imgsz=896,
        epochs=25,
        patience=7,
        batch=16,
        device=0,
        workers=4,
        project="runs_yolo",
        name=fold_name,
        exist_ok=True,
        seed=42,
        cos_lr=True,
        warmup_epochs=2,
        lr0=0.01,
        lrf=0.01,
        # conservative augs
        mosaic=0.0, mixup=0.0,
        hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
        fliplr=0.5, flipud=0.0,
        degrees=5.0, translate=0.05, scale=0.15, shear=0.0,
    )
    print(f"{fold_name} done in {time.time()-t0:.1f}s -> {res.save_dir}", flush=True)

print("Prepared. After cache completes, run sequentially:", flush=True)
print("train_fold(fold_yaml_paths[0], 'fold1_s896_e25p7')", flush=True)
print("train_fold(fold_yaml_paths[1], 'fold2_s896_e25p7')", flush=True)
print("train_fold(fold_yaml_paths[2], 'fold3_s896_e25p7')", flush=True)